## Login to Hugging Face

In [1]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")
login(
    token=token, # ADD YOUR TOKEN HERE
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/pathfinder/.cache/huggingface/token
Login successful


In [2]:
model_name = "waktaverse-gemma-ko-7b-it" # ADD YOUR MODEL NAME HERE
username = "PathFinderKR"  # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_name}"  # repository id

## Downloads

In [3]:
#!pip install huggingface_hub
#!pip install transformers
#!pip install bitsandbytes
#!pip install peft
#!pip install trl
#!pip install accelerate
#!pip install datasets
#!pip install scikit-learn
#!pip install packaging
#!pip install ninja
#!pip install flash-attn --no-build-isolation

## Imports

In [4]:
# pytorch
import torch

# huggingface
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# datasets
from datasets import load_dataset

## Device

In [5]:
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Hyperparameters

In [6]:
# Tokenizer arguments
max_length = 512 # maximum length of the text that can go to the model
padding = "max_length" # padding strategy: "longest", "max_length", "do_not_pad"
truncation = True # truncate the text if it exceeds the maximum length

# model arguments
max_new_tokens=500 # maximum number of tokens to generate

# mixed precision
dtype = torch.bfloat16 # data type

# quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # load model in 4-bit
    bnb_4bit_compute_dtype=dtype, # compute in (data type)
    bnb_4bit_quant_type="nf4", # quantize to 4-bit
    bnb_4bit_use_doulbe_quant=False # use double quantization
)

# LoRA configuration
lora_config = LoraConfig(
    task_type = "CAUSAL_LM", # task type
    r = 8, # rank
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"], # target modules
    lora_alpha = 16, # alpha
    lora_dropout = 0.1, # dropout
    bias="none", # bias
)

# training arguments
training_args = TrainingArguments(
    output_dir="./results", # output directory
    logging_dir="./logs", # logging directory
    save_strategy="epoch", # save strategy
    logging_strategy="steps", # logging strategy
    logging_steps=10, # logging steps
    save_total_limit=1, # save total limit
    
    learning_rate=2e-5, # learning rate
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=1, # training batch size
    per_device_eval_batch_size=1, # evaluation batch size
    optim="adamw_torch", # optimizer
    weight_decay=0.1, # weight decay
    lr_scheduler_type="cosine", # learning rate scheduler
    seed=42 # seed
)

# SFTTrainer arguments
max_seq_length = 512 # maximum sequence length

## Model

In [7]:
# Model List

# gemma variants
# "google/gemma-7b-it" // downloaded
# "PathFinderKR/waktaverse-gemma-ko-7b-it"
# "beomi/gemma-ko-7b"

# llama2 variants
# "meta-llama/Llama-2-7b-chat-hf" // downloaded
# "PathFinderKR/waktaverse-Llama-2-ko-7b-it"
# "beomi/KoAlpaca-Polyglot-5.8B" // downloaded
# "beomi/open-llama-2-ko-7b"

# solar variants
# "chihoonlee10/T3Q-ko-solar-dpo-v4.0"

In [8]:
model_id = "google/gemma-7b-it"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    attn_implementation="flash_attention_2",
    torch_dtype=dtype,
    quantization_config=quantization_config
)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Dataset

In [11]:
dataset = load_dataset("MarkrAI/KoCommercial-Dataset")

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 175454
    })
})

In [13]:
dataset["train"][0]

{'input': '',
 'instruction': '보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.',
 'output': '모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.'}

In [14]:
def preprocess_function(examples):
    # Concatenate the 'instruction' and 'output' fields for each example in the batch
    concatenated_texts = [instruction + ' ' + output for instruction, output in zip(examples['instruction'], examples['output'])]
    # Tokenize the concatenated texts
    return tokenizer(concatenated_texts, padding=padding, truncation=truncation, max_length=max_length)

dataset = dataset.map(preprocess_function, batched=True)

In [15]:
print(dataset["train"][0])

{'input': '', 'instruction': '보드 게임 스피너는 $A$, $B$, $C$로 표시된 세 부분으로 나뉩니다. 스피너가 $A$에 떨어질 확률은 $\\frac{1}{3}$이고, 스피너가 $B$에 떨어질 확률은 $\\frac{5}{12}$입니다.  스피너가 $C$에 착륙할 확률은 얼마입니까? 답을 공통 분수로 표현하세요.', 'output': '모든 가능한 결과의 확률의 합이 1$이므로, 스피너가 $C$에 착륙할 확률을 구하려면 스피너가 $A$와 $B$에 착륙할 확률을 1$에서 빼야 합니다. 이를 방정식으로 쓸 수 있습니다: $P(C) = 1 - P(A) - P(B)$. P(A) = \\frac{1}{3}$, $P(B) = \\frac{5}{12}$라는 것을 알고 있으므로 이 값을 방정식에 대입하여 단순화할 수 있습니다. 결과는 다음과 같습니다: P(C) = 1 - \\frac{1}{3} - frac{5}{12} = \\frac{12}{12} - frac{4}{12} - frac{5}{12} = \\frac{3}{12}$. 분자와 분모를 $3$로 나누면 이 분수를 줄일 수 있습니다: P(C) = \\frac{1}{4}$입니다.', 'input_ids': [2, 237036, 237135, 144280, 32275, 238810, 239632, 236214, 697, 235280, 5750, 697, 235305, 5750, 697, 235288, 235323, 236375, 100280, 236569, 238602, 48740, 43761, 238304, 26291, 38585, 255221, 12957, 235265, 32275, 238810, 239632, 236361, 697, 235280, 235323, 236179, 235248, 243433, 236770, 239574, 69781, 243156, 236648, 1467, 2552, 235282, 235274, 1214, 235304, 1208, 224907, 235269, 32275

## Inference before Fine-Tuning

In [16]:
# Chat Template
def generate_response(prompt):
    messages = [{"role": "user", "content": prompt }]
    chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(chat, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=input_ids.to(model.device), max_new_tokens=max_new_tokens)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [17]:
#prompt = "Write me a poem about Machine Learning."
prompt = "머신러닝에 대한 시를 써주세요."

In [18]:
response = generate_response(prompt)
print(response)

user
머신러닝에 대한 시를 써주세요.
model
**머신러닝의 시**

데이터가 쌓여진 바늘,
모형이 학습해 숨어진 규칙.
알고리즘이 작동하여,
정보를 추출하고 패턴을 발견.

인공지식이라는 마법,
데이터를 분석하고 문제 해결.
가벼운 망설이를 혐의,
새로운 가능성을 열어 갑니다.

사회를 바꾸는 기술,
정보의 전달에 영향을 미친다.
진료, 예측, 의료,
머신러닝은 모든 분야에 영향을 미친다.

따라서 머신러닝에 대한 희생감,
지식의 발달에 대한 기대.
미래를 위한 강력한 도움,
세상을 더 나은 곳으로 바꾸는 힘.


## Prompt Engineering

In [19]:
#text = "Write me a poem about Machine Learning"
text = "머신러닝에 대한 시를 써주세요."

In [20]:
prompt = f"""
You are Waktaverse-Gemma, a Korean language model, capable of performing various natural language processing tasks such as text generation, question answering, summarization, and more. 

Please respond to the following text delimited by triple backticks in Korean.
'''{text}'''
Use Korean only.
"""

In [21]:
response = generate_response(prompt)
print(response)

user
You are Waktaverse-Gemma, a Korean language model, capable of performing various natural language processing tasks such as text generation, question answering, summarization, and more. 

Please respond to the following text delimited by triple backticks in Korean.
'''머신러닝에 대한 시를 써주세요.'''
Use Korean only.
model
** Waktaverse-Gemma의 응답:**

머신 러닝에 대한 시를 써드립니다.
소박한 알고리즘,
데이터를 학습합니다.
모든 사물을 예측합니다.
세상을 변화시키는 강력한력.


## Supervised Fine-Tuning(LoRA)

In [22]:
def formatting_func(example):
    text = (f"instruction: {example['instruction'][0]}\n"
            f"output: {example['output'][0]}")
    return [text]

In [23]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    train_dataset=dataset["train"],
    formatting_func=formatting_func
)

/home/pathfinder/anaconda3/envs/torch-env/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
trainer.train()

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,6.281600
20,5.714500
30,5.023100
40,4.837300
50,3.677200
60,3.878400
70,3.317400
80,2.876700
90,2.845400
100,2.293800


TrainOutput(global_step=352, training_loss=2.614435675469312, metrics={'train_runtime': 1381.4238, 'train_samples_per_second': 0.255, 'train_steps_per_second': 0.255, 'total_flos': 5199232443371520.0, 'train_loss': 2.614435675469312, 'epoch': 2.0})

In [25]:
trainer.save_model(model_name)

## Inference after Fine-Tuning

In [26]:
#text = "Write me a poem about Machine Learning"
text = "머신러닝에 대한 시를 써주세요."

In [27]:
prompt = f"""
You are Waktaverse-Gemma, a Korean language model, capable of performing various natural language processing tasks such as text generation, question answering, summarization, and more. 

Please respond to the following text delimited by triple backticks in Korean.
'''{text}'''
Use Korean only.
"""

In [28]:
response = generate_response(prompt)
print(response)

user
You are Waktaverse-Gemma, a Korean language model, capable of performing various natural language processing tasks such as text generation, question answering, summarization, and more. 

Please respond to the following text delimited by triple backticks in Korean.
'''머신러닝에 대한 시를 써주세요.'''
Use Korean only.
model
머신러닝은 컴퓨터 과학의 한 분야로, 인공지식을 학습하는 능력을 가진 컴퓨터 프로그램입니다. 텍스트, 이미지, 음악, 영상 등 다양한 형태의 데이터를 분석하고 학습하여 새로운 정보를 생성하는 능력을 가집니다. 머신러닝은 인공지식의 한 분야로, 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을 축적하는 과정을 컴퓨터가 수행하는 것을 목표로 합니다. 머신러닝은 인간이 지식을

## Upload Model

In [29]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.float16,
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, model_name)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [30]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [31]:
# Push model and tokenizer to Hugging Face Hub
model.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)
tokenizer.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)

README.md:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PathFinderKR/waktaverse-gemma-ko-7b-it/commit/ea032b3f7d6546fd043981ebce7d2eafc014664d', commit_message='Upload tokenizer', commit_description='', oid='ea032b3f7d6546fd043981ebce7d2eafc014664d', pr_url=None, pr_revision=None, pr_num=None)